## Data preparation
 Original dataset is in TSV format and values are coded in a single string.
 Raw Dataset has "wide" format.
 This notebook parses the "key" string into additional columns and changes table format into "long". Null values, originally coded as ":" will be changed to `null`

In [71]:
!pip -q install pandas

In [72]:
imigration_data_path = "Immigration_raw.csv"
integration_data_path = ""

In [73]:
import pandas as pd

immigration_ds = pd.read_csv(imigration_data_path, sep=",")

columns = immigration_ds.columns

# address empty cells and "p", "pe" ect. tags
immigration_ds = immigration_ds.map(lambda x: x.strip() if isinstance(x, str) else x)


def extract_leading_int(val):
    if isinstance(val, str):
        parts = val.strip().split()
        if parts and parts[0].isdigit():
            return int(parts[0])
    return val


for col in columns[1:]:
    immigration_ds[col] = immigration_ds[col].apply(extract_leading_int)
immigration_ds = immigration_ds.replace(":", pd.NA)

# parse dimention_key
columns_clean = [
    "freq",
    "age",
    "agedef",
    "c_birth",
    "unit",
    "sex",
    "geo",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
]

In [74]:
immigration_ds.head(5)

,"freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD",2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,AT",4085,4093,4723,4827,<NA>,4130,4286,<NA>,<NA>,<NA>,<NA>
1,"A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,BE",1407,953,1017,1004,<NA>,1364,1514,<NA>,<NA>,<NA>,<NA>
2,"A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,BG",743,577,559,586,<NA>,1413,1659,<NA>,<NA>,<NA>,<NA>
3,"A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,CH",2758,2766,2876,3109,<NA>,3209,3198,<NA>,<NA>,<NA>,<NA>
4,"A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,CZ",155,106,120,414,<NA>,736,1013,<NA>,<NA>,<NA>,<NA>


In [75]:
rows = []
for idx, row in immigration_ds.iterrows():
    complex_key = row.iloc[0]
    parsed = complex_key.split(",")
    rest = row.iloc[1:].tolist()
    all_values = parsed + rest
    rows.append(all_values)

new_ds = pd.DataFrame(rows, columns=columns_clean)

In [76]:
new_ds.head(5)

,freq,age,agedef,c_birth,unit,sex,geo,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,AT,4085,4093,4723,4827,<NA>,4130,4286,<NA>,<NA>,<NA>,<NA>
1,A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,BE,1407,953,1017,1004,<NA>,1364,1514,<NA>,<NA>,<NA>,<NA>
2,A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,BG,743,577,559,586,<NA>,1413,1659,<NA>,<NA>,<NA>,<NA>
3,A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,CH,2758,2766,2876,3109,<NA>,3209,3198,<NA>,<NA>,<NA>,<NA>
4,A,TOTAL,COMPLET,CC5_13_FOR_X_IS,NR,F,CZ,155,106,120,414,<NA>,736,1013,<NA>,<NA>,<NA>,<NA>


In [77]:
# Drop "unit" as it has only one value
new_ds = new_ds.drop(columns=["unit", "freq"])

In [9]:
age_band_map = {
    'Y_LT5': 'Y00-04',
    'Y5-9': 'Y05-09',
    'Y10-14': 'Y10-14',
    'Y15-19': 'Y15-19',
    'Y15-64': 'Y15-64',
    'Y20-24': 'Y20-24',
    'Y25-29': 'Y25-29',
    'Y30-34': 'Y30-34',
    'Y35-39': 'Y35-39',
    'Y40-44': 'Y40-44',
    'Y45-49': 'Y45-49',
    'Y50-54': 'Y50-54',
    'Y55-59': 'Y55-59',
    'Y60-64': 'Y60-64',
    'Y65-69': 'Y65-69',
    'Y70-74': 'Y70-74',
    'Y75-79': 'Y75-79',
    'Y80-84': 'Y80-84',
    'Y85-89': 'Y85-89',
    'Y90-94': 'Y90-94',
    'Y95-99': 'Y95-99',
    'TOTAL': 'TOTAL',
    'UNK': 'UNK',
    'Y_GE100': 'Y_GE100',
    'Y_GE65': 'Y_GE65',
    'Y_GE85': 'Y_GE85',
    'Y_LT15': 'Y_LT15'
}



In [ ]:
new_ds.age = new_ds.age.map(age_band_map)

In [79]:
new_ds

,age,agedef,c_birth,sex,geo,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,4085,4093,4723,4827,<NA>,4130,4286,<NA>,<NA>,<NA>,<NA>
1,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,BE,1407,953,1017,1004,<NA>,1364,1514,<NA>,<NA>,<NA>,<NA>
2,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,BG,743,577,559,586,<NA>,1413,1659,<NA>,<NA>,<NA>,<NA>
3,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,CH,2758,2766,2876,3109,<NA>,3209,3198,<NA>,<NA>,<NA>,<NA>
4,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,CZ,155,106,120,414,<NA>,736,1013,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67399,Y00-04,REACH,UNK,T,NO,0,0,0,0,0,0,0,0,0,0,0
67400,Y00-04,REACH,UNK,T,PL,0,22,173,170,0,0,0,114,<NA>,64,27
67401,Y00-04,REACH,UNK,T,PT,0,0,0,0,0,0,0,0,0,0,0
67402,Y00-04,REACH,UNK,T,SE,5,9,13,31,29,24,15,6,21,6,7


### Now invert format from wide to long

In [80]:
long_columns = ["age", "agedef", "c_birth", "sex", "geo", "year", "nr"]
rows = []

years = ["2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
expected_length = len(years) * new_ds.shape[0]
i, one_perc = 0, int(expected_length / 100)
for idx, row in new_ds.iterrows():
    if i > one_perc:
        print(f"Processing row {idx}/{len(new_ds)} {(idx * 100 / len(new_ds)):.0f} %")
        i = 0
    for year in years:
        values = row.iloc[:5].tolist()
        nr = row[year]
        values.extend([year, nr])
        rows.append(values)
    i += 1

long_df = pd.DataFrame(rows, columns=long_columns)

Processing row 7415/67404 11 %
Processing row 14830/67404 22 %
Processing row 22245/67404 33 %
Processing row 29660/67404 44 %
Processing row 37075/67404 55 %
Processing row 44490/67404 66 %
Processing row 51905/67404 77 %
Processing row 59320/67404 88 %
Processing row 66735/67404 99 %


In [81]:
long_df.head(5)

,age,agedef,c_birth,sex,geo,year,nr
0,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,2013,4085
1,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,2014,4093
2,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,2015,4723
3,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,2016,4827
4,TOTAL,COMPLET,CC5_13_FOR_X_IS,F,AT,2017,<NA>


In [82]:
long_df.to_csv("Immigration.csv", index=False)

In [83]:
long_df.c_birth.unique()

array(['CC5_13_FOR_X_IS', 'CC5_15_FOR', 'CC8_22_FOR', 'CC9_23_FOR',
       'EFTA_FOR', 'EU27_2020_FOR', 'EU28_FOR', 'EXT_FOR_HDI',
       'EXT_FOR_HDI_H', 'EXT_FOR_HDI_L', 'EXT_FOR_HDI_M',
       'EXT_FOR_HDI_VH', 'NAT', 'NEU27_2020_FOR', 'NEU28_FOR', 'TOTAL',
       'UNK'], dtype=object)

In [84]:
long_df.geo.unique()

array(['AT', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI',
       'FR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MT',
       'NL', 'NO', 'PL', 'RO', 'SE', 'SI', 'SK', 'UK', 'CY', 'EU27_2020',
       'ME', 'MK', 'PT', 'MD', 'UA'], dtype=object)

In [85]:
long_df.age

0          TOTAL
1          TOTAL
2          TOTAL
3          TOTAL
4          TOTAL
           ...  
741439    Y00-04
741440    Y00-04
741441    Y00-04
741442    Y00-04
741443    Y00-04
Name: age, Length: 741444, dtype: object

# Europe population ds

In [13]:
import pandas as pd

df = pd.read_csv("europe_population_raw.csv")

In [14]:
df.columns

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency',
       'unit', 'Unit of measure', 'sex', 'Sex', 'age', 'Age class', 'geo',
       'Geopolitical entity (reporting)', 'TIME_PERIOD', 'Time', 'OBS_VALUE',
       'Observation value', 'OBS_FLAG',
       'Observation status (Flag) V2 structure', 'CONF_STATUS',
       'Confidentiality status (flag)'],
      dtype='object')

In [15]:
df = df[['geo', 'TIME_PERIOD', 'OBS_VALUE', "age", "sex"]].rename(columns={"TIME_PERIOD": "year", "OBS_VALUE": "population"}) 

In [16]:

df.age = df.age.map(age_band_map)


In [17]:
df.to_csv("europe_population.csv", index=False)

In [90]:
long_df.geo.unique()

array(['AT', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI',
       'FR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MT',
       'NL', 'NO', 'PL', 'RO', 'SE', 'SI', 'SK', 'UK', 'CY', 'EU27_2020',
       'ME', 'MK', 'PT', 'MD', 'UA'], dtype=object)

In [91]:
long_df.c_birth.unique()

array(['CC5_13_FOR_X_IS', 'CC5_15_FOR', 'CC8_22_FOR', 'CC9_23_FOR',
       'EFTA_FOR', 'EU27_2020_FOR', 'EU28_FOR', 'EXT_FOR_HDI',
       'EXT_FOR_HDI_H', 'EXT_FOR_HDI_L', 'EXT_FOR_HDI_M',
       'EXT_FOR_HDI_VH', 'NAT', 'NEU27_2020_FOR', 'NEU28_FOR', 'TOTAL',
       'UNK'], dtype=object)

In [92]:
long_df.age.unique()

array(['TOTAL', 'UNK', 'Y10-14', 'Y15-19', 'Y15-64', 'Y20-24', 'Y25-29',
       'Y30-34', 'Y35-39', 'Y40-44', 'Y45-49', 'Y05-09', 'Y50-54',
       'Y55-59', 'Y60-64', 'Y65-69', 'Y70-74', 'Y75-79', 'Y80-84',
       'Y85-89', 'Y90-94', 'Y95-99', 'Y_GE100', 'Y_GE65', 'Y_GE85',
       'Y_LT15', 'Y00-04'], dtype=object)